In [244]:
using CUDA
using ProgressMeter
using DelimitedFiles
include("../src/forces_func.jl")
include("../src/struct_data.jl")
include("../src/test.jl")
include("../src/forces.jl")
include("../src/neighbor.jl")
include("../src/run_event.jl")
include("../src/sphere.jl")

@make_struct_func Cubic

force_func (generic function with 1 method)

In [245]:
parameters = Model(
    Cubic(
        μ₁       = 0.003,
        rₘᵢₙ     = 2,
        rₘₐₓ     = 3
    ),
    Contractile(
        fₚ       = 0.001
    ),
    Time(
        t_f      = 50000, 
        dt       = 0.1
    ),
    Neighbor(
        n_knn    = 50,
        nn       = 12
    ),
    Geometry(
        R_agg    = 9,
        position = [
            0.0    0.0    0.0;
            0.0    0.0    0.0
        ]
    ),
    Simulation(
        n_text   = 200,
        path     = "After-Eq.xyz"
    )
)

## Var before func
println("\n Loading Force Function")
CUDA.@time force(r) =  force_func(parameters.Force::Cubic, r)


 Loading Force Function
  0.000039 seconds (17 CPU allocations: 1.237 KiB)


force (generic function with 1 method)

In [246]:
function generate_random_packaging(R_agg, m::Model)
    R_sup = R_agg
    while true
        println("R_Agg = $(R_sup)")
        println("Random Packaging")
        X = sphere(R_sup)

        # Generating Struct
        Agg = Aggregate(Neighbor(50, 12), PositionCell(X))

        # Simulation One Aggregate
        one_aggregate(false, m, Cubic(1.0, 2.0, 3.0), Agg)

        Size = Float32(
                (findmax(Agg.Position.X[:,1])[1] - findmin(Agg.Position.X[:,1])[1])/2 + 1
            ) 

        println("---------")
        println("R_Agg = $(Size)")
        println("---------")

        if Size < R_agg
            R_sup += + 0.5
        else
            break
        end
    end

    open("$(R_agg).xyz", "w") do f
        write(f, "$(size(X, 1))\n")
        write(f, "Random Packaging ($(R_agg)).xyz\n")
        writedlm(f,hcat(repeat([1.0],size(X, 1)), Matrix(X)), ' ')
    end
    
end

generate_random_packaging (generic function with 1 method)

In [247]:
R_agg = 9
generate_random_packaging(R_agg, parameters)

R_Agg = 9
Random Packaging


Progress:  79%|███████████████████▋     |  ETA: 0:00:03

Not Enought
Finding Equilibrium in one Aggregate


Progress:  36%|████████▉                |  ETA: 0:04:19error in running finalizer: InterruptException()
maybe_collect at /buildworker/worker/package_linux64/build/src/julia_threads.h:313 [inlined]
jl_gc_pool_alloc at /buildworker/worker/package_linux64/build/src/gc.c:1204
default_stream at /home/nicomosty/.julia/packages/CUDA/DfvRa/lib/cudadrv/stream.jl:32 [inlined]
unsafe_finalize! at /home/nicomosty/.julia/packages/CUDA/DfvRa/src/array.jl:99
_jl_invoke at /buildworker/worker/package_linux64/build/src/gf.c:2237 [inlined]
jl_apply_generic at /buildworker/worker/package_linux64/build/src/gf.c:2419
jl_apply at /buildworker/worker/package_linux64/build/src/julia.h:1703 [inlined]
run_finalizer at /buildworker/worker/package_linux64/build/src/gc.c:278
jl_gc_run_finalizers_in_list at /buildworker/worker/package_linux64/build/src/gc.c:365
run_finalizers at /buildworker/worker/package_linux64/build/src/gc.c:394
jl_gc_collect at /buildworker/worker/package_linux64/build/src/gc.c:3260
maybe_coll

In [ ]:
Y = Float32.(readdlm("$(R_agg).xyz")[3:end,2:end]) |> cu

321×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.0         0.0        0.0
 -5.87014    -2.19887   -2.14153
 -1.07578    -3.33097   -3.82132
 -3.46719     5.92836   -2.34629
  4.80081    -4.77961    1.16843
  6.59362    -3.45945    3.7898
 -1.11985     2.44693    6.36483
 -0.0         0.0        7.48295
 -0.297133   -0.679201   2.39659
 -3.21578     5.49849   -0.186075
 -0.552346   -4.89389   -6.59279
 -3.26762     0.135987  -2.38055
 -1.08376     1.85307   -2.8737
  ⋮                     
 -7.15489     1.0199     3.67842
 -7.64374    -2.86323   -3.73559
  1.40942     1.45119   -5.69001
  7.65715    -3.08668    1.42395
  0.378712    5.34038   -7.16684
 -5.95124    -0.347992  -6.50798
  7.14001     4.88474    2.40166
  6.63108    -1.92968   -0.201742
 -4.54411     1.61557    5.72578
 -0.0154408  -1.24627    6.14852
 -3.46549     7.57224    3.23759
  5.44594     6.86274    1.51107

In [ ]:
Size = Float32(
    (findmax(Y[:,1])[1] - findmin(Y[:,1])[1])/2 + 1
) 

9.482686f0

In [ ]:
run(`paraview 9.xyz`)

Process(`paraview 9.xyz`, ProcessExited(0))